### Compare ###

In the previous chapter we have performed predictions but these were rather uneducated (to say the least). The next step is to use some method to quantify our error, and then use this error in a clever way to adjust our weights in such a way so as to reduce it.
In fact, coming up with a good method to quantify our error is one of the hardest things to achieve.

In the beginning we will only use the MSE (Mean Squared Error) however it should be noted that many more methods exist. The __compare__ step, will give us an idea of "how much we missed" but on its own, this is not enough to achieve learning.


### Learn ###
Learning is all about figuring out how much each weight contributed to the error. This will tell us how much each weight will need to change in order to reduce the error. This "blame game" is called __Gradient Descent__.



In [1]:
knob_weight = 0.5
input = 0.5
goal_pred = 0.8
pred = input * knob_weight
error = (pred - goal_pred) ** 2
print(error)

0.30250000000000005


-  __goal_pred__: Similar to _input_, this number is recorded somewhere in the real world but it is something that it is usually hard to observe (unlike _input_). This is why we want to predict it using the _input_
-  Notice that we square the error. This is done for two reasons:
    -  Firstly, it makes all errors positive (otherwise, they would cancel each other out when added). This might not sound important but it actually is since later we are going to be working with multiple input -> prediction pairs and what we will want to minimise will be the average error (Mean SE). 
    -  Secondly, it amplifies larger errors (>1) and reduces smaller ones (<1). This offers an "error prioritisation" where we pay more attention to the larger errors.    

### The simplest form of Neural Learning ###
At the end of the day, learning is about one thing: __adjusting the weights up/down so that the _error_ is reduced__. The simplest way in choosing the direction that the weight should move, is try both, and choose the one that actually reduces the error. 
After doing this over and over, eventually the error is reduced to 0.
<img src="images/4.simplest_form_of_learning.PNG">

In [7]:
weight = 0.1
lr = 0.01

def neural_network(input, weight):
    prediction = input * weight
    return prediction

win_or_lose_binary = [1]
number_of_toes = [8.5]

input = number_of_toes[0]
true = win_or_lose_binary[0]

pred = neural_network(input, weight)
error = (pred - true) ** 2
print ("ERROR: {}".format(error))

# Make a prediction with a higher weight:
pred_up = neural_network(input, weight + lr)
error_up = (pred_up - true) ** 2
print ("ERROR (UP): {}".format(error_up))

# Make a prediction with a lower weight:
pred_dn = neural_network(input, weight - lr)
error_dn = (pred_dn - true) ** 2
print ("ERROR (DOWN): {}".format(error_dn))

if (error > error_dn or error > error_up):
    if error_dn < error_up:
        weight -= lr
    
    if error_dn > error_up:
        weight += lr

print("Updated weight to: {}".format(weight))


ERROR: 0.022499999999999975
ERROR (UP): 0.004224999999999993
ERROR (DOWN): 0.05522499999999994
Updated weight to: 0.11


We could easily modify the code above to run in a loop so that the error is reduced at every iteration. However, this method will have some issues:
-  We have to predict multiple times in order to perform a single weight update since we don't know the correct direction.
-  Sometimes it is difficult to guess learning rate correctly; a big value will cause the error to oscillate since everytime it will overshoot in the opposite direction. A (too) small value will cause the error to decrease very slowly which is inefficient.

__Gradient Descent__ allows us to calculate __both__ the _amount_ and _direction_ that the weight should be modified with.

In [8]:
weight = 0.5
goal_pred = 0.8
input = 0.5

for iteration in range(20):
    pred = input * weight
    error = (pred - goal_pred) ** 2
    direction_and_amount = (pred - goal_pred) * input
    weight = weight - direction_and_amount
    
    print("Error: {} | Prediction: {}".format(error, pred))

Error: 0.30250000000000005 | Prediction: 0.25
Error: 0.17015625000000004 | Prediction: 0.3875
Error: 0.095712890625 | Prediction: 0.49062500000000003
Error: 0.05383850097656251 | Prediction: 0.56796875
Error: 0.03028415679931642 | Prediction: 0.6259765625
Error: 0.0170348381996155 | Prediction: 0.669482421875
Error: 0.00958209648728372 | Prediction: 0.70211181640625
Error: 0.005389929274097089 | Prediction: 0.7265838623046875
Error: 0.0030318352166796153 | Prediction: 0.7449378967285156
Error: 0.0017054073093822882 | Prediction: 0.7587034225463867
Error: 0.0009592916115275371 | Prediction: 0.76902756690979
Error: 0.0005396015314842384 | Prediction: 0.7767706751823426
Error: 0.000303525861459885 | Prediction: 0.7825780063867569
Error: 0.00017073329707118678 | Prediction: 0.7869335047900676
Error: 9.603747960254256e-05 | Prediction: 0.7902001285925507
Error: 5.402108227642978e-05 | Prediction: 0.7926500964444131
Error: 3.038685878049206e-05 | Prediction: 0.7944875723333098
Error: 1.70926

__Things to note__:
-  The __direction_and_amount__ term is in reality the first derivative of the MSE w.r.t the weight. It is important to remember that the MSE is a quadratic. (The sign of the derivative is the direction, and its steepness is the amount). This means that if the gradient is +ve, we are on the right side of the bell. Therefore by reducing the weight, we will reduce the error. 
If the gradient is -ve, it means that we are on the left side of the bell -> thus by increasing the weights we decrease the error.
This is why in the next line: <br>
`weight = weight - direction_and_amount` 
<br>
we reverse the sign of this scalar - __the weights should move in the opposite direction of the gradient to reach the minimum error!__
The gradient not only provides us of the (reverse) direction to move, but also of the amount:<br><br>
-  The term `(pred - goal_pred)` is the "pure error". The higher this value is, the more we adjust the weights. This makes sense if we think about the quadratic error again. Higher values of this, suggest that we are far from the minimum -> so we can afford to make larger corrections. As we repeat the process, and this value is reduced, so does the amount of correction which prevents us from overshooting (not completely, wait a bit!).<br><br>

-  The fact that the gradient includes the `input` term helps us address three major edge cases for which "pure error" does not suffice.
    -  Firstly, if the input is 0, then the weight adjustment is also 0. This makes sense as there is little to learn. This is called __"stopping"__.
    -  Secondly, notice that when the error is large, the the weight is further from the the value that would give the minimum error (again, think quadratic). 
    

<img src="images/5.quadratic_error_graph.PNG">


In [10]:
weight, goal_pred, input = (0.0, 0.8, 1.1)
for iteration in range(4):
    print("-----\nWeight:{}".format(weight))
    pred = input * weight
    error = (pred - goal_pred) ** 2
    delta = pred - goal_pred
    weight_delta = delta * input
    weight = weight - weight_delta
    print("Error: {}, Prediction: {}".format(str(error), str(pred)))
    print("Delta: {}, Weight Delta: {}".format(str(delta), str(weight_delta)))

-----
Weight:0.0
Error: 0.6400000000000001, Prediction: 0.0
Delta: -0.8, Weight Delta: -0.8800000000000001
-----
Weight:0.8800000000000001
Error: 0.02822400000000005, Prediction: 0.9680000000000002
Delta: 0.16800000000000015, Weight Delta: 0.1848000000000002
-----
Weight:0.6951999999999999
Error: 0.0012446784000000064, Prediction: 0.76472
Delta: -0.03528000000000009, Weight Delta: -0.0388080000000001
-----
Weight:0.734008
Error: 5.4890317439999896e-05, Prediction: 0.8074088
Delta: 0.007408799999999993, Weight Delta: 0.008149679999999992


__Note that these two lines of code:__ <br>
`
pred = input * weight
error = (pred - goal_pred) ** 2
`<br><br>
__could be rewritten in one as:__ <br>
` error = ((weight * input) - goal_pred) ** 2`<br><br>
__Why is this important?__<br>
Well, we have managed to discover a _direct relationship_ between the _weight_ and the _error_. This is what enables us to know the correct amount and direction that we should move to decrease the error (through the power of calculus and derivatives). If you think about it, a derivative is nothing more than the direction and amount that one variable will move if you change another variable. 

### Divergence ###
Although we have managed to find a way to adjust our weights to bring down the error, this doesn't always work. Consider the example below:


In [12]:
weight = 0.5
goal_pred = 0.8
# Change the input value to 2
input = 2 
for iteration in range(20):
    pred = input * weight
    error = (pred - goal_pred) ** 2
    delta = pred - goal_pred
    weight_delta = input * delta
    weight = weight - weight_delta
    print("Error: {}, Prediction: {}".format(str(error), str(pred)))


Error: 0.03999999999999998, Prediction: 1.0
Error: 0.3599999999999998, Prediction: 0.20000000000000018
Error: 3.2399999999999984, Prediction: 2.5999999999999996
Error: 29.159999999999986, Prediction: -4.599999999999999
Error: 262.4399999999999, Prediction: 16.999999999999996
Error: 2361.959999999998, Prediction: -47.79999999999998
Error: 21257.639999999978, Prediction: 146.59999999999994
Error: 191318.75999999983, Prediction: -436.5999999999998
Error: 1721868.839999999, Prediction: 1312.9999999999995
Error: 15496819.559999991, Prediction: -3935.799999999999
Error: 139471376.03999993, Prediction: 11810.599999999997
Error: 1255242384.3599997, Prediction: -35428.59999999999
Error: 11297181459.239996, Prediction: 106288.99999999999
Error: 101674633133.15994, Prediction: -318863.79999999993
Error: 915071698198.4395, Prediction: 956594.5999999997
Error: 8235645283785.954, Prediction: -2869780.599999999
Error: 74120807554073.56, Prediction: 8609344.999999996
Error: 667087267986662.1, Predicti

Notice that at each iteration the update overcorrects and oscillates from +ve to -ve, everytime overshooting even more. This is because __input__ is part of the derivative, and even when the error is small, a big enough __input__ will cause the update rule to have a high value:
<img src="images/6.Gradient_Descent_Divergence.PNG">

### Introducing _α_ ###
The simplest way to correct for this, is to multiply our update value by fraction each time, before updating the weight. This will case our algorithm to converge more slowly to (almost) 0 error, but it will also prevent divergence issues.
We only need to change one line of code:
    

In [15]:
weight = 0.5
goal_pred = 0.8
input = 2 
alpha = 0.1

for iteration in range(20):
    pred = input * weight
    error = (pred - goal_pred) ** 2
    delta = pred - goal_pred
    weight_delta = input * delta
    weight = weight - (alpha * weight_delta)
    print("Error: {}, Prediction: {}".format(str(error), str(pred)))

Error: 0.03999999999999998, Prediction: 1.0
Error: 0.0144, Prediction: 0.92
Error: 0.005183999999999993, Prediction: 0.872
Error: 0.0018662400000000014, Prediction: 0.8432000000000001
Error: 0.0006718464000000028, Prediction: 0.8259200000000001
Error: 0.00024186470400000033, Prediction: 0.815552
Error: 8.70712934399997e-05, Prediction: 0.8093312
Error: 3.134566563839939e-05, Prediction: 0.80559872
Error: 1.1284439629823931e-05, Prediction: 0.803359232
Error: 4.062398266736526e-06, Prediction: 0.8020155392
Error: 1.4624633760252567e-06, Prediction: 0.8012093235200001
Error: 5.264868153690924e-07, Prediction: 0.8007255941120001
Error: 1.8953525353291194e-07, Prediction: 0.8004353564672001
Error: 6.82326912718715e-08, Prediction: 0.8002612138803201
Error: 2.456376885786678e-08, Prediction: 0.8001567283281921
Error: 8.842956788836216e-09, Prediction: 0.8000940369969153
Error: 3.1834644439835434e-09, Prediction: 0.8000564221981492
Error: 1.1460471998340758e-09, Prediction: 0.800033853318889

It should be pointed out that in the previous example the learning rate alpha was chosen arbitrarily and it just worked. This usually requires some trial and error. Usual values are [0.1, 0.01, 0.001, 0.0001]. 